In [1]:
# !pip freeze > requirements.txt
# !pip install -r requirements.txt

In [2]:
import os
import json
from datetime import datetime
import pytz
from collections import Counter

import torch
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from torchvision import models, transforms
from tqdm import tqdm
import numpy as np
from PIL import Image
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from torchsummary import summary

import warnings
warnings.filterwarnings("ignore")

# Preprocessing

In [3]:
class PathLabelProcessor:
    def __init__(self, 
                 base_path,
                 folder_name,
                 pet_type,
                 lesion,
                 devices,
                 symptom):
        self.base_path = base_path
        self.folder_name = folder_name
        self.pet_type = pet_type
        self.lesion = lesion
        self.devices = devices
        self.symptom = symptom
        
        self.label_images()
        
    def find_folders_by_name(self):
        matching_folders = []

        for root, dirs, files in os.walk(self.base_path):
            for dir_name in dirs:
                if self.folder_name in dir_name:
                    folder_path = os.path.join(root, dir_name)
                    matching_folders.append(folder_path)

        for folder_path in matching_folders:
            print(folder_path)
            
        return matching_folders

    def find_image_json_pairs(self):
        image_paths = []
        json_paths = []

        for folder_path in self.find_folders_by_name():
            for root, dirs, files in os.walk(folder_path):
                for image_file in filter(lambda x: x.lower().endswith(('jpg', 'png')), files):
                    image_path = os.path.join(root, image_file)
                    json_file = f"{os.path.splitext(image_path)[0]}.json"
                    if os.path.isfile(json_file):
                        image_paths.append(image_path)
                        json_paths.append(json_file)

        print(f'Total images: {len(image_paths)}, Total JSON files: {len(json_paths)}')
        
        return image_paths, json_paths

    def label_images(self):
        self.labeled_image_paths = []

        for image_path, json_path in zip(*self.find_image_json_pairs()):
            with open(json_path) as f:
                data = json.load(f)
                
            is_symptomatic = data['label']['label_disease_lv_3'] in self.symptom and data['label']['label_disease_nm'] == self.lesion

            if data['images']['meta']['device'] in self.devices:
                if self.pet_type in os.path.dirname(image_path).lower():
                    self.labeled_image_paths.append((image_path, 0 if is_symptomatic else 1))
                else:
                    self.labeled_image_paths.append((image_path, 1))
        
        self.symptomatic_count = len(Counter(item for item in self.labeled_image_paths if item[1] == 0))
        self.asymptomatic_count = len(Counter(item for item in self.labeled_image_paths if item[1] == 1))

        print(f'Total cases: {len(self.labeled_image_paths)}')
        print(f'Number of symptomatic cases: {self.symptomatic_count}, Number of asymptomatic cases: {self.asymptomatic_count}')

class ImageDataset(Dataset):
    def __init__(self, labeled_image_paths, transform):
        self.labeled_image_paths = labeled_image_paths
        self.transform = transform

    def __len__(self):
        return len(self.labeled_image_paths)

    def __getitem__(self, idx):
        image_path, label = self.labeled_image_paths[idx]

        image = Image.open(image_path)
        image = self.transform(image)
        label = torch.tensor([label], dtype=torch.float32)

        return image, label

class DataLoaderMaker:
    def __init__(self,
                 dataset,
                 batch_size,
                 train_ratio=0,
                 num_workers=None):
        self.dataset = dataset
        self.train_ratio = train_ratio
        self.batch_size = batch_size
        self.num_workers = num_workers

        if train_ratio:
            self.split_and_make_dataloader()
        else:
            self.dataloader = self.make_dataloader(self.dataset)

    def make_dataloader(self, dataset, shuffle=False):
        dataloader = DataLoader(dataset=dataset,
                                batch_size=self.batch_size,
                                shuffle=shuffle,
                                num_workers=self.num_workers,
                                pin_memory=True)
        
        self.inspect_data(dataloader)
        
        return dataloader

    def split_and_make_dataloader(self):
        train_size = int(len(self.dataset) * self.train_ratio)
        test_size = len(self.dataset) - train_size
        train_dataset, test_dataset = random_split(self.dataset, [train_size, test_size])

        self.train_loader = self.make_dataloader(train_dataset, shuffle=True)
        self.test_loader = self.make_dataloader(test_dataset, shuffle=True)

    def inspect_data(self, dataloader):
        print("Inspecting Data...")

        class_counts = {}
        for _, labels in dataloader:
            labels = labels.type(torch.int)
            for label in labels.tolist():
                label_tuple = tuple(label)
                class_counts[label_tuple] = class_counts.get(label_tuple, 0) + 1

        print("- Class Counts:")
        for class_label, count in class_counts.items():
            print(f"  Class {class_label}: {count} samples")

In [4]:
%%time
base_path = 'eye/Train'
folder_name = '일반'
'''
['유']
dog: 안검염, 안검종양, 안검 내반증, 유루증, 색소침착성각막염, 핵경화, 결막염
cat: 안검염, 결막염, 각막부골편, 비궤양성각막염, 각막궤양
['상', '하']
dog: 각막질환, 비궤양성 각막질환
['초기', '비성숙', '성숙']
dog: 백내장
'''
pet_type = '개'
lesion = '안검염'
devices = ['스마트폰', '일반카메라']
symptom = ['유']

processor = PathLabelProcessor(base_path=base_path,
                                folder_name=folder_name,
                                pet_type=pet_type,
                                lesion=lesion,
                                devices=devices,
                                symptom=symptom)

labeled_image_paths = processor.labeled_image_paths

eye/Train/고양이/안구/일반
eye/Train/개/안구/일반
Total images: 199816, Total JSON files: 199816
Total cases: 98646
Number of symptomatic cases: 1139, Number of asymptomatic cases: 97507
CPU times: user 6.03 s, sys: 2.46 s, total: 8.49 s
Wall time: 9.48 s


In [5]:
%%time
transform = transforms.Compose([transforms.Resize((240, 240)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

dataset = ImageDataset(labeled_image_paths=labeled_image_paths,
                       transform=transform)

CPU times: user 366 µs, sys: 0 ns, total: 366 µs
Wall time: 376 µs


In [6]:
%%time
batch_size = 96
num_workers = os.cpu_count()
train_ratio = 0.8

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers,
                              train_ratio=train_ratio)

Inspecting Data...


- Class Counts:
  Class (1,): 78011 samples
  Class (0,): 905 samples
Inspecting Data...
- Class Counts:
  Class (1,): 19496 samples
  Class (0,): 234 samples
CPU times: user 19.6 s, sys: 16.6 s, total: 36.2 s
Wall time: 55.2 s


# Modeling

In [7]:
class ModelTrainer:
    def __init__(self, 
                 model,
                 device,
                 train_dataloader,
                 valid_dataloader,
                 loss_fn,
                 optimizer,
                 scheduler,
                 lesion):
        self.device = device
        self.model = model.to(self.device)
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.best_f1_score = 0.0
        korea = pytz.timezone('Asia/Seoul')
        now = datetime.now(korea)
        start_time = now.strftime('%Y%m%d-%H%M%S')
        self.name = f'{start_time}_{lesion}.pth'
        self.writer = SummaryWriter(log_dir=f'runs/{self.name}')

    def calculate_f1_score(self, predicted, labels):
        return f1_score(labels, predicted, average='binary')

    def calculate_auc_roc(self, predicted, labels):
        return roc_auc_score(labels, predicted)

    def train_one_epoch(self, epoch, num_epochs):
        self.model.train()
        total_loss = 0.0
        correct = 0
        total = 0
    
        for step, (inputs, labels) in enumerate(tqdm(self.train_dataloader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')):
            inputs, labels = inputs.to(self.device), labels.to(self.device)
    
            outputs = self.model(inputs)
    
            loss = self.loss_fn(outputs, labels)
            total_loss += loss.item()
    
            probs = torch.sigmoid(outputs)
            predicted = (probs > 0.5).int()
    
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
            loss.backward()
    
            self.optimizer.step()
            self.optimizer.zero_grad()
    
        self.scheduler.step()
        self.writer.add_scalar('LearningRate', self.scheduler.get_last_lr()[0], epoch)
    
        avg_loss = total_loss / len(self.train_dataloader)
        accuracy = correct / total
        self.writer.add_scalar('Loss/train', avg_loss, epoch)        
        self.writer.add_scalar('Accuracy/train', accuracy, epoch)

    def eval_one_epoch(self, epoch):
        self.model.eval()
        total_loss = 0.0
        correct = 0
        total = 0
        all_predicted = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in self.valid_dataloader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
        
                outputs = self.model(inputs)
        
                loss = self.loss_fn(outputs, labels)
                total_loss += loss.item()
        
                probs = torch.sigmoid(outputs)
                predicted = (probs > 0.5).int()
        
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
                all_predicted.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        avg_loss = total_loss / len(self.valid_dataloader)
        accuracy = correct / total
        self.writer.add_scalar('Loss/valid', avg_loss, epoch)
        self.writer.add_scalar('Accuracy/valid', accuracy, epoch)
        
        current_f1_score = self.calculate_f1_score(np.array(all_predicted), np.array(all_labels))
        current_auc_roc = self.calculate_auc_roc(np.array(all_predicted), np.array(all_labels))
        
        self.writer.add_scalar('F1 Score/valid', current_f1_score, epoch)
        self.writer.add_scalar('AUC-ROC/valid', current_auc_roc, epoch)
        
        if current_f1_score > self.best_f1_score:
            self.best_f1_score = current_f1_score
            torch.save({
                'model': self.model,
                'optimizer_state_dict': self.optimizer.state_dict(),
                'scheduler_state_dict': self.scheduler.state_dict()
            }, self.name)

    def train(self, num_epochs):
        for epoch in range(num_epochs):
            self.train_one_epoch(epoch, num_epochs)
            self.eval_one_epoch(epoch)

In [8]:
model = models.efficientnet_b1()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 1)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.001)

trainer = ModelTrainer(model=model,
                       device=device,
                       train_dataloader=data_loader.train_loader,
                       valid_dataloader=data_loader.test_loader,
                       loss_fn=loss_fn,
                       optimizer=optimizer,
                       scheduler=scheduler)

In [9]:
trainer.train(30)

Epoch 20/30:  43%|████▎     | 350/823 [02:59<03:59,  1.97batch/s]

# Evaluation

In [16]:
class ModelTester:
    def __init__(self, path, device, dataloader):
        self.device = device
        self.dataloader = dataloader
        self.load_model(path)
        self.evaluate()

    def load_model(self, path):
        checkpoint = torch.load(path)
        
        self.model = checkpoint['model']
        self.model.to(self.device)
        print("Loaded Model Information:")
        summary(self.model, input_size=(3, 240, 240))
        
        print("Optimizer State:")
        print(checkpoint['optimizer_state_dict'])
        
        print("Scheduler State:")
        print(checkpoint['scheduler_state_dict'])

    def classify(self):
        self.model.eval()
        predictions = []
        labels = []
        probabilities = []

        with torch.no_grad():
            for inputs, targets in tqdm(self.dataloader):
                inputs = inputs.to(self.device)
                targets = targets.to(self.device)
                outputs = self.model(inputs)
                
                probs = torch.nn.functional.softmax(outputs, dim=1)
                _, predicted = torch.max(outputs, 1)

                predictions.extend(predicted.cpu().numpy())
                labels.extend(targets.cpu().numpy())
                probabilities.extend(probs.max(dim=1).values.cpu().numpy())

        return predictions, labels, probabilities

    def calculate_prob_stats(self, probabilities):
        probabilities = np.array(probabilities)
        min_probs = np.min(probabilities)
        max_probs = np.max(probabilities)
        std_probs = np.std(probabilities)
        mean_probs = np.mean(probabilities)

        return min_probs, max_probs, std_probs, mean_probs

    def evaluate(self):
        predictions, labels, probabilities = self.classify()
        cm = confusion_matrix(labels, predictions)
        accuracy = accuracy_score(labels, predictions)
        precision = precision_score(labels, predictions)
        recall = recall_score(labels, predictions)
        f1 = f1_score(labels, predictions)

        min_probs, max_probs, std_probs, mean_probs = self.calculate_prob_stats(probabilities)

        print("Evaluation Results:")
        print(f"Confusion Matrix:\n{cm}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"Min Probability: {min_probs:.4f}")
        print(f"Max Probability: {max_probs:.4f}")
        print(f"Standard Deviation of Probabilities: {std_probs:.4f}")
        print(f"Mean Probability: {mean_probs:.4f}")

In [17]:
%%time
base_path = 'eye/Valid'
folder_name = '일반'
pet_type = '개'
lesion = '안검염'
devices = ['스마트폰', '일반카메라']
symptom = ['유']

processor = PathLabelProcessor(base_path=base_path,
                                folder_name=folder_name,
                                pet_type=pet_type,
                                lesion=lesion,
                                devices=devices,
                                symptom=symptom)

labeled_image_paths = processor.labeled_image_paths

eye/Valid/고양이/안구/일반
eye/Valid/개/안구/일반
Total images: 24976, Total JSON files: 24976
Total cases: 13808
Number of symptomatic cases: 138, Number of asymptomatic cases: 13670
CPU times: user 1.16 s, sys: 252 ms, total: 1.41 s
Wall time: 1.41 s


In [18]:
%%time
transform = transforms.Compose([transforms.Resize((240, 240)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

dataset = ImageDataset(labeled_image_paths=labeled_image_paths,
                       transform=transform)

batch_size = 32
num_workers = os.cpu_count()

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

path = '20231205-092138_안검염.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ModelTester(path=path, device=device, dataloader=data_loader.dataloader)

Inspecting Data...
- Class Counts:
  Class (1,): 13670 samples
  Class (0,): 138 samples


TypeError: 'EfficientNet' object is not subscriptable

In [14]:
%%time
dataset = ImageDataset(labeled_image_paths=[item for item in labeled_image_paths if item[1] == 0],
                       transform=transform)

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

ModelTester(path=path, device=device, dataloader=data_loader.dataloader)

Inspecting Data...
- Class Counts:
  Class (0,): 138 samples
Loaded Model Information:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 120, 120]             864
       BatchNorm2d-2         [-1, 32, 120, 120]              64
              SiLU-3         [-1, 32, 120, 120]               0
            Conv2d-4         [-1, 32, 120, 120]             288
       BatchNorm2d-5         [-1, 32, 120, 120]              64
              SiLU-6         [-1, 32, 120, 120]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
              SiLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
          Sigmoid-11             [-1, 32, 1, 1]               0
SqueezeExcitation-12         [-1, 32, 120, 120]               0
           Conv2

100%|██████████| 5/5 [00:01<00:00,  4.04it/s]

Evaluation Results:
Confusion Matrix:
[[138]]
Accuracy: 1.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Min Probability: 1.0000
Max Probability: 1.0000
Standard Deviation of Probabilities: 0.0000
Mean Probability: 1.0000
CPU times: user 604 ms, sys: 1.53 s, total: 2.13 s
Wall time: 2.54 s


In [15]:
%%time
dataset = ImageDataset(labeled_image_paths=[item for item in labeled_image_paths if item[1] == 1],
                       transform=transform)

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

ModelTester(path=path, device=device, dataloader=data_loader.dataloader)

Inspecting Data...
- Class Counts:
  Class (1,): 13670 samples
Loaded Model Information:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 120, 120]             864
       BatchNorm2d-2         [-1, 32, 120, 120]              64
              SiLU-3         [-1, 32, 120, 120]               0
            Conv2d-4         [-1, 32, 120, 120]             288
       BatchNorm2d-5         [-1, 32, 120, 120]              64
              SiLU-6         [-1, 32, 120, 120]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
              SiLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
          Sigmoid-11             [-1, 32, 1, 1]               0
SqueezeExcitation-12         [-1, 32, 120, 120]               0
           Con

100%|██████████| 428/428 [00:20<00:00, 20.86it/s]


Evaluation Results:
Confusion Matrix:
[[    0     0]
 [13670     0]]
Accuracy: 0.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Min Probability: 1.0000
Max Probability: 1.0000
Standard Deviation of Probabilities: 0.0000
Mean Probability: 1.0000
CPU times: user 25.3 s, sys: 6.95 s, total: 32.3 s
Wall time: 29.1 s


In [ ]:
class PreModelTester:
    def __init__(self, path, device, dataloader):
        self.device = device
        self.dataloader = dataloader
        self.model = models.vgg16_bn(pretrained=True)
        self.load_model(path)
        self.evaluate()

    def load_model(self, path):
        self.model = models.vgg16_bn(pretrained=True)
        nr_filters = self.model.classifier[0].in_features
        self.model.classifier = nn.Linear(nr_filters, 1)
        state_dict = torch.load(path, map_location=torch.device("cpu"))
        model_dict = self.model.state_dict()
        state_dict = {k: v for k, v in state_dict.items() if k in model_dict}
        model_dict.update(state_dict)
        self.model.load_state_dict(model_dict)
        self.model = self.model.to(self.device)
        
        print("Loaded Model Information:")
        summary(self.model, input_size=(3, 240, 240))

    def classify(self):
        self.model.eval()
        predictions = []
        labels = []
        probabilities = []

        with torch.no_grad():
            for inputs, targets in tqdm(self.dataloader):
                inputs = inputs.to(self.device)
                targets = targets.to(self.device)
                outputs = self.model(inputs)
                
                probs = torch.nn.functional.softmax(outputs, dim=1)
                _, predicted = torch.max(outputs, 1)

                predictions.extend(predicted.cpu().numpy())
                labels.extend(targets.cpu().numpy())
                probabilities.extend(probs.max(dim=1).values.cpu().numpy())

        return predictions, labels, probabilities

    def calculate_prob_stats(self, probabilities):
        probabilities = np.array(probabilities)
        min_probs = np.min(probabilities)
        max_probs = np.max(probabilities)
        std_probs = np.std(probabilities)
        mean_probs = np.mean(probabilities)

        return min_probs, max_probs, std_probs, mean_probs

    def evaluate(self):
        predictions, labels, probabilities = self.classify()
        cm = confusion_matrix(labels, predictions)
        accuracy = accuracy_score(labels, predictions)
        precision = precision_score(labels, predictions)
        recall = recall_score(labels, predictions)
        f1 = f1_score(labels, predictions)

        min_probs, max_probs, std_probs, mean_probs = self.calculate_prob_stats(probabilities)

        print("Evaluation Results:")
        print(f"Confusion Matrix:\n{cm}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"Min Probability: {min_probs:.4f}")
        print(f"Max Probability: {max_probs:.4f}")
        print(f"Standard Deviation of Probabilities: {std_probs:.4f}")
        print(f"Mean Probability: {mean_probs:.4f}")

In [ ]:
%%time
dataset = ImageDataset(labeled_image_paths=labeled_image_paths,
                       transform=transform)

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

path = 'pre_eye.pt'

PreModelTester(path=path, device=device, dataloader=data_loader.dataloader)

In [ ]:
%%time
dataset = ImageDataset(labeled_image_paths=[item for item in labeled_image_paths if item[1] == 0],
                       transform=transform)

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

PreModelTester(path=path, device=device, dataloader=data_loader.dataloader)

In [ ]:
%%time
dataset = ImageDataset(labeled_image_paths=[item for item in labeled_image_paths if item[1] == 1],
                       transform=transform)

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

PreModelTester(path=path, device=device, dataloader=data_loader.dataloader)

# Hyperparameter Tuning